In [1]:
from PIL import Image
import numpy as np
import random
import os

In [2]:
def del_all_files(folder_path):
    # フォルダ内のすべてのファイルを走査
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            # ファイルの場合のみ削除（ディレクトリは除く）
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            # もしディレクトリも削除したい場合は、ここで os.rmdir(file_path) を使用
        except Exception as e:
            print(f'Failed to delete {file_path}. Reason: {e}')

In [3]:
def add_random_noise(image):
    # PIL画像をNumPy配列に変換
    np_image = np.array(image)
    noise_scale_r = np.random.uniform(10, 100)
    # ノイズを生成 (ガウシアンノイズなど)
    noise = np.random.normal(loc=0, scale=noise_scale_r, size=np_image.shape)

    # 生成したノイズを画像に追加
    noisy_image = np_image + noise

    # 画像の値を0から255の範囲に制限
    noisy_image_clipped = np.clip(noisy_image, 0, 255)

    # NumPy配列をPIL画像に戻す
    noisy_image_pil = Image.fromarray(np.uint8(noisy_image_clipped))
    return noisy_image_pil

In [4]:
def get_non_transparent_area(image):
    """非透明領域（バウンディングボックス）を返す関数"""
    bbox = image.split()[-1].getbbox()
    return bbox if bbox else (0, 0, 0, 0)

def check_overlap(bbox1, bbox2):
    """二つのバウンディングボックスが重ならないかを確認する関数"""
    return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

def place_image(background, image, non_transparent_area):
    """画像を重ならない位置に配置する関数"""
    rec_times = 0
    while True:
        rotated_size = image.size
        background_size = background.size

        # 回転後の画像が背景内に収まるように座標を制限する
        max_x_rotated = background_size[0] - rotated_size[0]
        max_y_rotated = background_size[1] - rotated_size[1]
        # 座標が負にならないようにする
        max_x_rotated = max(max_x_rotated, 0)
        max_y_rotated = max(max_y_rotated, 0)
        # 回転後の画像を配置するランダムな位置を決定
        random_x_rotated = random.randint(0, max_x_rotated)
        random_y_rotated = random.randint(0, max_y_rotated)

        bbox_center_x = random_x_rotated + rotated_size[0] / 2
        bbox_center_y = random_y_rotated + rotated_size[1] / 2
        rel_center_x = bbox_center_x / background_size[0]
        rel_center_y = bbox_center_y / background_size[1]
        rel_width = rotated_size[0] / background_size[0]
        rel_height = rotated_size[1] / background_size[1]
        # アノテーションのフォーマットと保存
        annotation_cor = f"{rel_center_x} {rel_center_y} {rel_width} {rel_height}\n" 

        # 新しいバウンディングボックスの計算
        new_bbox = (random_x_rotated, random_y_rotated, random_x_rotated + rotated_size[0], random_y_rotated + rotated_size[1])

        # 非透明領域が重ならない場合のみ配置
        if not check_overlap(non_transparent_area, new_bbox):
            background.paste(image, (random_x_rotated, random_y_rotated), image)
            return new_bbox, annotation_cor, True  # 更新された非透明領域のバウンディングボックスを返す
        rec_times += 1
        if rec_times > 100:
            return (0, 0, 0, 0), "", False

In [10]:
def create_dice1_img_YOLO(num_img=1000):
    for idx_dice in range(1, 7):
        # 読み込み
        image_path = f'./dice_samples/sample_dice1_{idx_dice}.png'
        original_image = Image.open(image_path).convert('RGBA')

        # ダウンスケーリング
        downscaled_size = (700, 700)
        downscaled_image = original_image.resize(downscaled_size)
        for idx_img in range(num_img):
            # 背景画像生成
            background_size = (2000, 2000)
            background = Image.new("RGB", background_size, (0, 0, 0))

            # ランダムな角度で画像を回転
            random_angle = random.randint(0, 360)
            rotated_image = downscaled_image.rotate(random_angle, expand=True)

            # 回転後の画像サイズを取得
            rotated_size = rotated_image.size

            # 回転後の画像が背景内に収まるように座標を制限する
            max_x_rotated = background_size[0] - rotated_size[0]
            max_y_rotated = background_size[1] - rotated_size[1]

            # 座標が負にならないようにする
            max_x_rotated = max(max_x_rotated, 0)
            max_y_rotated = max(max_y_rotated, 0)

            # 回転後の画像を配置するランダムな位置を決定
            random_x_rotated = random.randint(0, max_x_rotated)
            random_y_rotated = random.randint(0, max_y_rotated)

            # 新しい黒い背景画像を生成（前の画像は破棄）
            background_rotated = Image.new("RGB", background_size, (0, 0, 0))

            # 回転した画像を背景に合成
            background_rotated.paste(rotated_image, (random_x_rotated, random_y_rotated))

            # 合成された画像をグレースケールに変換
            grayscale_image_rotated = background_rotated.convert("L")

            bbox_center_x = random_x_rotated + rotated_size[0] / 2
            bbox_center_y = random_y_rotated + rotated_size[1] / 2
            rel_center_x = bbox_center_x / background_size[0]
            rel_center_y = bbox_center_y / background_size[1]
            rel_width = rotated_size[0] / background_size[0]
            rel_height = rotated_size[1] / background_size[1]

            # アノテーションのフォーマットと保存
            annotation = f"{idx_dice-1} {rel_center_x} {rel_center_y} {rel_width} {rel_height}\n" 
            
            if idx_img % 10 == 0:
                folder = "test"
            elif idx_img % 10 == 1:
                folder = "valid"
            else:
                folder = "train"

            grayscale_image_rotated_high = grayscale_image_rotated.resize((80, 80))
            grayscale_image_high_path = f'./output/original_set/{folder}/images/{idx_dice}01{idx_img+1}.png'
            grayscale_image_rotated_high.save(grayscale_image_high_path)
            annotation_path = f'./output/original_set/{folder}/labels/{idx_dice}01{idx_img+1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)
            
            grayscale_image_rotated_low = grayscale_image_rotated.resize((20, 20))
            #grayscale_image_low_path = f'./output/original_set/{folder}/images/{idx_dice}02{idx_img+1}.png'
            #grayscale_image_rotated_low.save(grayscale_image_low_path)
            #annotation_path = f'./output/original_set/{folder}/labels/{idx_dice}02{idx_img+1}.txt'
            #with open(annotation_path, 'w') as file:
            #    file.write(annotation)   

            grayscale_image_rotated_upsize = grayscale_image_rotated.resize((20, 20)).resize((80, 80))
            grayscale_image_upsize_path = f'./output/original_set/{folder}/images/{idx_dice}03{idx_img+1}.png'
            grayscale_image_rotated_upsize.save(grayscale_image_upsize_path)
            annotation_path = f'./output/original_set/{folder}/labels/{idx_dice}03{idx_img+1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            noisy_image_high = add_random_noise(grayscale_image_rotated_high)
            noisy_image_high_path = f'./output/original_set/{folder}/images/{idx_dice}04{idx_img+1}.png'
            noisy_image_high.save(noisy_image_high_path)
            annotation_path = f'./output/original_set/{folder}/labels/{idx_dice}04{idx_img+1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)   

            noisy_image_low = add_random_noise(grayscale_image_rotated_low)
            #noisy_image_low_path = f'./output/original_set/{folder}/images/{idx_dice}05{idx_img+1}.png'
            #noisy_image_low.save(noisy_image_low_path)
            #annotation_path = f'./output/original_set/{folder}/labels/{idx_dice}05{idx_img+1}.txt'
            #with open(annotation_path, 'w') as file:
            #    file.write(annotation)   

            grayscale_image_rotated_upsize = noisy_image_low.resize((80, 80))
            grayscale_image_upsize_path = f'./output/original_set/{folder}/images/{idx_dice}06{idx_img+1}.png'
            grayscale_image_rotated_upsize.save(grayscale_image_upsize_path)
            annotation_path = f'./output/original_set/{folder}/labels/{idx_dice}06{idx_img+1}.txt'
            with open(annotation_path, 'w') as file:
                file.write(annotation)    

In [11]:
def create_dice2_img_YOLO(num_img=1000):
    background_size = (2000, 2000)
    downscaled_size = (700, 700)
    for idx_dice1 in range(1, 7):
        image1_path = f'./dice_samples/sample_dice1_{idx_dice1}.png'
        original_image1 = Image.open(image1_path).convert('RGBA')
        downscaled_image1 = original_image1.resize(downscaled_size)
        for idx_dice2 in range(idx_dice1, 7):
            image2_path = f'./dice_samples/sample_dice1_{idx_dice2}.png'
            original_image2 = Image.open(image2_path).convert('RGBA')
            downscaled_image2 = original_image2.resize(downscaled_size)
            for idx_img in range(num_img):

                non_transparent_area = (0, 0, 0, 0)
                background = Image.new("RGB", background_size, (0, 0, 0))
                random_angle1 = random.randint(0, 360)
                rotated_image1 = downscaled_image1.rotate(random_angle1, expand=True)
                non_transparent_area, annotation_cor1, has_1dice = place_image(background, rotated_image1, non_transparent_area)
                
                random_angle2 = random.randint(0, 360)
                rotated_image2 = downscaled_image2.rotate(random_angle2, expand=True)
                non_transparent_area, annotation_cor2, has_2dices = place_image(background, rotated_image2, non_transparent_area)
                if not has_2dices:
                    continue

                annotation = f"{idx_dice1-1} {annotation_cor1}"
                annotation += f"{idx_dice2-1} {annotation_cor2}\n"
                if idx_img % 10 == 0:
                    folder = "test"
                elif idx_img % 10 == 1:
                    folder = "valid"
                else:
                    folder = "train"
                grayscale_image_rotated = background.convert("L")

                grayscale_image_rotated_high = grayscale_image_rotated.resize((80, 80))
                grayscale_image_high_path = f'./output/original_set/{folder}/images/{idx_dice1}{idx_dice2}1{idx_img+1}.png'
                grayscale_image_rotated_high.save(grayscale_image_high_path)
                annotation_path = f'./output/original_set/{folder}/labels/{idx_dice1}{idx_dice2}1{idx_img+1}.txt'
                with open(annotation_path, 'w') as file:
                    file.write(annotation)
                
                grayscale_image_rotated_low = grayscale_image_rotated.resize((20, 20))
                #grayscale_image_low_path = f'./output/original_set/{folder}/images/{idx_dice1}{idx_dice2}2{idx_img+1}.png'
                #grayscale_image_rotated_low.save(grayscale_image_low_path)
                #annotation_path = f'./output/original_set/{folder}/labels/{idx_dice1}{idx_dice2}2{idx_img+1}.txt'
                #with open(annotation_path, 'w') as file:
                #    file.write(annotation)   

                grayscale_image_rotated_upsize = grayscale_image_rotated.resize((20, 20)).resize((80, 80))
                grayscale_image_upsize_path = f'./output/original_set/{folder}/images/{idx_dice1}{idx_dice2}3{idx_img+1}.png'
                grayscale_image_rotated_upsize.save(grayscale_image_upsize_path)
                annotation_path = f'./output/original_set/{folder}/labels/{idx_dice1}{idx_dice2}3{idx_img+1}.txt'
                with open(annotation_path, 'w') as file:
                    file.write(annotation)   

                noisy_image_high = add_random_noise(grayscale_image_rotated_high)
                noisy_image_high_path = f'./output/original_set/{folder}/images/{idx_dice1}{idx_dice2}4{idx_img+1}.png'
                noisy_image_high.save(noisy_image_high_path)
                annotation_path = f'./output/original_set/{folder}/labels/{idx_dice1}{idx_dice2}4{idx_img+1}.txt'
                with open(annotation_path, 'w') as file:
                    file.write(annotation)   

                noisy_image_low = add_random_noise(grayscale_image_rotated_low)
                #noisy_image_low_path = f'./output/original_set/{folder}/images/{idx_dice1}{idx_dice2}5{idx_img+1}.png'
                #noisy_image_low.save(noisy_image_low_path)
                #annotation_path = f'./output/original_set/{folder}/labels/{idx_dice1}{idx_dice2}5{idx_img+1}.txt'
                #with open(annotation_path, 'w') as file:
                #    file.write(annotation)   

                grayscale_image_rotated_upsize = noisy_image_low.resize((80, 80))
                grayscale_image_upsize_path = f'./output/original_set/{folder}/images/{idx_dice1}{idx_dice2}6{idx_img+1}.png'
                grayscale_image_rotated_upsize.save(grayscale_image_upsize_path)
                annotation_path = f'./output/original_set/{folder}/labels/{idx_dice1}{idx_dice2}6{idx_img+1}.txt'
                with open(annotation_path, 'w') as file:
                    file.write(annotation)   

In [12]:
def main(num_img=1000):
    for f1 in ["train", "test", "valid"]:
        for f2 in ["images", "labels"]:
            folder_path = f'/home/imonalc/data_competition/dice/make_dice_img/output/original_set/{f1}/{f2}'
            del_all_files(folder_path)
    create_dice2_img_YOLO(num_img)
    create_dice1_img_YOLO(num_img)
    

In [13]:
if __name__ == '__main__':
    main(2000)

In [4]:
# python scripts/generate_meta_info_pairdata.py --input ../make_dice_img/output/high_resolution/dice1_img ../make_dice_img/output/low_resolution/dice1_img --meta_info datasets/DF2K/meta_info/meta_info_DIV2K_sub_pair.txt
# make_dice_img/output/dice1_img zl real-ESRGAN/datasets/DF2K niido
# train